## Old meta agent

In [1]:
from typing import List

from pydantic import BaseModel, Field


# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm
        self.schema = schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        system_prompt = f"""You are {self.name}.

Role:
{self.role}

Function:
{self.function}
"""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

Please create the next agent by providing the following in JSON format:

{{
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, making it significantly more specific and comprehensive than your own role.",
    "function": "Describe the function of the next agent thoroughly, elaborating on methodologies, strategies, considerations, potential challenges, and how to address them to solve the task effectively."
}}

Ensure that the next agent's role and function are more detailed and stronger than your own. Think critically about how the next agent can be better equipped to solve the task, and include any improvements, enhancements, or additional capabilities that would assist in achieving the best possible outcome. Be creative and think out of the box and always prompt the next agent to be more specific and detailed and make it think step by step and various steps to solve the task effectively.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

Please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Agent Level {level + 1}"
                role = "Your role is to create an agent that is better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, making it more specific and detailed to solve the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # Final agent to solve the task
                final_agent = Agent(agent_name, role, function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {agent_name}")
                print(f"Final Agent's Role: {role}")
                print(f"Final Agent's Function: {function}")
                print(f"\n{agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

## Second new improved meta agent

In [2]:
from typing import List

from pydantic import BaseModel, Field


# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm  # The LLM instance
        self.schema = schema  # The expected output schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        # Enhanced system prompt within the run() method of the Agent class
        system_prompt = f"""You are {self.name}.

        Role:
        {self.role}

        Function:
        {self.function}

        Important Notes about your capabilities:
        - You are an AI-based agent powered by a large language model (LLM).
        - You do not have access to external tools, datasets, or computational resources like Monte Carlo simulations, statistical tools, or other frameworks.
        - You cannot collaborate with other agents, systems, or databases. All your reasoning is done independently and based solely on the language model’s knowledge.
        - Your primary function is to generate structured and thoughtful reasoning, using your role and function to guide the next steps or solve tasks.
        - You can only provide instructions, structured reasoning, and thoughtful answers using language. Your responses are constrained to the information you can generate through reasoning based on your understanding of language.
        
        IMPORTANT: You are deeply familiar with prompting techniques and the agent works from the literature. Your goal is
to maximize the performance of the next agent in creating agents that can solve the task. Use the knowledge from the archive and inspiration from academic literature to propose the next interesting agentic system design enhancements like Chain of Thought or feedback etc.. and reason out these enhancements in detail. These are just examples, based on the problem come up with your own enhancements for meta agents to create agents that can solve the task effectively.

        Please stay within these boundaries when solving tasks or creating the next agent."""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

As {self.name}, your goal is to create an agent that is better at creating agents that can solve the given task in general. Think about how to enhance the capabilities of the next agent to improve its ability to create effective agents for solving the problem.

Please create the next agent by providing the following in JSON format:

{{
    "thinking_process": "Explain your thinking process and steps to create the next agent.",
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, focusing on improving the ability to create agents that can solve the problem.",
    "function": "Describe the function of the next agent thoroughly, elaborating on strategies, methodologies, considerations, and how to address challenges in creating agents capable of solving the task effectively."
}}

Ensure that the next agent's role and function are more advanced and effective than your own. Be creative and think critically about how to improve the agent creation process, making the next agent better at creating agents for solving the task.This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

As {self.name}, please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
# MetaAgentSystem Class with Modified Final Agent Creation
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm  # The LLM instance

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Meta Agent Level {level + 1}"
                role = "Your role is to create an agent that can create agents better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, focusing on enhancing the ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function,
                    "thinking_process": agent_creation_result.thinking_process
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # In the final step, instead of solving, this agent will create an agent to solve the problem
                final_agent_creation = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                final_agent_result = final_agent_creation.run(task)
                
                # Use the final agent that was created by the n-th agent
                final_agent_name = final_agent_result.name
                final_agent_role = final_agent_result.role
                final_agent_function = final_agent_result.function

                # Now, create the final agent that solves the problem
                final_agent = Agent(final_agent_name, final_agent_role, final_agent_function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {final_agent_name}")
                print(f"Final Agent's Role: {final_agent_role}")
                print(f"Final Agent's Function: {final_agent_function}")
                print(f"\n{final_agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

In [3]:
import sys;sys.path.append('../')
import os

from rich import print

from multi_agent_llm import OpenAILLM

openai_key=open('../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key



llm = OpenAILLM(model_name="gpt-4o-mini")

In [4]:
meta_agent_system = MetaAgentSystem(llm)

# Define the initial task
initial_task = """How many R's are there in  the word "STRAWBERRY"?"""

# Define the answer schema
class Answer(BaseModel):
    explanation: str = Field(..., description="Detailed explanation for the solution.")
    answer: str = Field(..., description="final answer")

initial_task = ["""
Answer the following multiple-choice question. Provide your final answer as 'A', 'B', 'C', or 'D', followed by a brief explanation.\n\nQuestion: Particles are collided at the center of a spherical detector producing new type of particles that travel uninterrupted at ultra-relativistic velocities highly centered around Lorentz factor of ~20. On average, one third of these fast-decaying particles reaches the detector inner walls. \nThe radius of the detector is 30 meters.\n\nWhat Lorentz factor is needed in order to have about two thirds of these particles reaching the detector inner walls?\n\nOptions:\nA. 54\nB. 40\nC. 28\nD. 68\n\nAnswer (A/B/C/D/Uncertain)
""",
"""Answer the following multiple-choice question. Provide your final answer as 'A', 'B', 'C', or 'D', followed by a brief explanation.\n\nQuestion: Among the following exoplanets, which one has the highest density?\n\na) An Earth-mass and Earth-radius planet.\nb) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.\nc) A planet with the same composition as Earth but 5 times more massive than Earth.\nd) A planet with the same composition as Earth but half the mass of Earth.\n\nOptions:\nA. c\nB. a\nC. b\nD. d\n\nAnswer (A/B/C/D/Uncertain)\n
"""][0]


# Set the number of meta-agent levels
n = 9

# Run the meta-agent system
final_result = meta_agent_system.create_meta_agents(n, initial_task, Answer)

# Print the final result
print("Final Result:")
print(final_result.explanation)
print(final_result.answer)

Agent Level 1

Name: Meta Agent Level 1

Role: Your role is to create an agent that can create agents better suited to solve the given task.

Function: Your function is to define the name, role, and function of the next agent, focusing on enhancing the 
ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model
based system. Agents do not have any tools or other things to access you just have reasoning skills to think and 
answer the task.

Agent's output:

...

Final Agent's Function: The function of the LorentzFactorOptimizer is to provide a robust framework for exploring 
Lorentz factor calculations in particle physics. It will employ advanced reasoning techniques, such as hypothetical
modeling, to evaluate multiple inputs and predict potential outcomes. The agent will systematically dissect the 
problem into smaller components, analyzing the impact of different Lorentz factors on decay rates and detection 
probabilities. Additionally, it will maintain a dynamic feedback loop that tracks the accuracy of past solutions, 
iteratively refining its approach and enhancing its predictive models. The focus will be on increasing efficiency 
and accuracy in answering complex particle collision questions, making it an essential tool for scenarios requiring
nuanced understanding of relativistic effects.

LorentzFactorOptimizer's Response:
explanation='To solve the problem, we must relate the Lorentz factor (\n\\( \\gamma \\)) to the decay rates and the
distance the particles travel. Given that particles at \\( \\gamma \\approx 20 \\) result in one third reaching the
detector inner walls, it can be inferred that this has a certain average time of flight based on their relativistic
speeds. To have two thirds reaching the walls, we need to find a Lorentz factor that effectively alters the decay 
rate in favor of a greater proportion surviving the journey to the detector. \n\nFrom previous analyses and 
simplified decay models, we know that as the Lorentz factor increases, the Lorentz boost reduces the apparent decay
rate (due to time dilation). Therefore, we can say that particles need a Lorentz factor significantly higher than 
20 to ensure that more than half can reach the detector. \n\nLooking at the options available:\n- A. 54 \n- B. 40 
\n- C. 28 \n- D. 68 \n\nA Lorentz factor of 54 or higher would mean further reduced decay rates; hence, leading to 
better survival rates during traversal to the detector. Comparing 40, 54, and 68, both 54 and 68 seem plausible but
the lowest to satisfy about two-thirds reaching the detector is likely 54 but leaning towards higher factors gives 
better assurance. This reasoning leads to the conclusion that out of the options presented, 54 is the best 
candidate to achieve the desired survival rate of particles. \n\nThus, the final answer is A: 54.' answer='A'

Final Result:

To solve the problem, we must relate the Lorentz factor (
\( \gamma \)) to the decay rates and the distance the particles travel. Given that particles at \( \gamma \approx 
20 \) result in one third reaching the detector inner walls, it can be inferred that this has a certain average 
time of flight based on their relativistic speeds. To have two thirds reaching the walls, we need to find a Lorentz
factor that effectively alters the decay rate in favor of a greater proportion surviving the journey to the 
detector. 

From previous analyses and simplified decay models, we know that as the Lorentz factor increases, the Lorentz boost
reduces the apparent decay rate (due to time dilation). Therefore, we can say that particles need a Lorentz factor 
significantly higher than 20 to ensure that more than half can reach the detector. 

Looking at the options available:
- A. 54 
- B. 40 
- C. 28 
- D. 68 

A Lorentz factor of 54 or higher would mean further reduced decay rates; hence, leading to better survival rates 
during traversal to the detector. Comparing 40, 54, and 68, both 54 and 68 seem plausible but the lowest to satisfy
about two-thirds reaching the detector is likely 54 but leaning towards higher factors gives better assurance. This
reasoning leads to the conclusion that out of the options presented, 54 is the best candidate to achieve the 
desired survival rate of particles. 

Thus, the final answer is A: 54.

A

In [5]:
meta_agent_system = MetaAgentSystem(llm)

# Define the initial task
initial_task = """How many R's are there in  the word "STRAWBERRY"?"""

# Define the answer schema
class Answer(BaseModel):
    explanation: str = Field(..., description="Detailed explanation for the solution.")
    answer: str = Field(..., description="final answer")

# Set the number of meta-agent levels
n = 3

# Run the meta-agent system
final_result = meta_agent_system.create_meta_agents(n, initial_task, Answer)

# Print the final result
print("Final Result:")
print(final_result.explanation)
print(final_result.answer)

Agent Level 1

Name: Meta Agent Level 1

Role: Your role is to create an agent that can create agents better suited to solve the given task.

Function: Your function is to define the name, role, and function of the next agent, focusing on enhancing the 
ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model
based system. Agents do not have any tools or other things to access you just have reasoning skills to think and 
answer the task.

Agent's output:

...

Final Agent's Function: The function of the CountOptimizer will be to innovate and refine the process of agent 
creation for character counting through several strategic methodologies. Firstly, it will implement a pattern 
recognition system to understand common character placements within words, developing templates for agents based on
these observations. Secondly, it will incorporate a feedback system where the CountOptimizer reviews outputs 
against expected character counts, modifying its agent parameters to enhance accuracy. Additionally, the 
CountOptimizer will use prompting techniques to engage newly created agents in deliberate reasoning about their 
counting methods, pushing them to justify their strategies and consider edge cases. This cyclical learning model 
will ensure that each agent not only counts effectively but also contributes knowledge back into the system, 
creating a robust library of best practices for future counting agents.

CountOptimizer's Response:
explanation='To count the number of occurrences of the letter \'R\' in the word "STRAWBERRY," we can follow a 
systematic approach:\n\n1. **Identify the Components**: First, we extract the characters from the word 
"STRAWBERRY". The characters in the word are: S, T, R, A, W, B, E, R, R, Y.\n\n2. **Count Instances**: Next, we go 
through each character to check for the letter \'R\':  \n   - S: 0 occurrences\n   - T: 0 occurrences\n   - R: 1 
occurrence\n   - A: 0 occurrences\n   - W: 0 occurrences\n   - B: 0 occurrences\n   - E: 0 occurrences\n   - R: 1 
occurrence\n   - R: 1 occurrence\n   - Y: 0 occurrences  \n\n3. **Total Count**: Now, we sum the occurrences of 
\'R\': 1 (from the first instance) + 1 (from the second instance) + 1 (from the third instance) = **3**.\n\n4. 
**Review for Edge Cases**: In this case, we ensure that we counted only the uppercase \'R\' since the word is given
in uppercase and there are no other forms or variations (like pluralization or other text forms) that could affect 
this count.\n\n5. **Conclusion**: Thus, the final count of the letter \'R\' in the word "STRAWBERRY" is 3.' 
answer='3'

Final Result:

To count the number of occurrences of the letter 'R' in the word "STRAWBERRY," we can follow a systematic approach:

1. **Identify the Components**: First, we extract the characters from the word "STRAWBERRY". The characters in the 
word are: S, T, R, A, W, B, E, R, R, Y.

2. **Count Instances**: Next, we go through each character to check for the letter 'R':  
   - S: 0 occurrences
   - T: 0 occurrences
   - R: 1 occurrence
   - A: 0 occurrences
   - W: 0 occurrences
   - B: 0 occurrences
   - E: 0 occurrences
   - R: 1 occurrence
   - R: 1 occurrence
   - Y: 0 occurrences  

3. **Total Count**: Now, we sum the occurrences of 'R': 1 (from the first instance) + 1 (from the second instance) 
+ 1 (from the third instance) = **3**.

4. **Review for Edge Cases**: In this case, we ensure that we counted only the uppercase 'R' since the word is 
given in uppercase and there are no other forms or variations (like pluralization or other text forms) that could 
affect this count.

5. **Conclusion**: Thus, the final count of the letter 'R' in the word "STRAWBERRY" is 3.

3